In [ ]:
!conda install -c conda-forge gdcm -y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import os
from pydicom import dcmread # for dcm files
import pydicom
import cv2
import csv

from sklearn.metrics import confusion_matrix, accuracy_score
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py
import gdcm
import PIL
import tensorflow as tf
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
image_paths = []
SOPs_train = []
import os
for dirname, _, filenames in os.walk('../input/rsna-str-pulmonary-embolism-detection/train'):
    for filename in filenames:
        temp = os.path.join(dirname, filename)
        image_paths.append(os.path.join(dirname, filename))
        SOPs_train.append(temp[len(temp)-16:len(temp)-4])
        

image_paths = np.asarray(image_paths)
SOPs_train = np.asarray(SOPs_train)

In [ ]:
with open ('../input/rsna-str-pulmonary-embolism-detection/train.csv') as csvfile:  
    readCSVFeatures = csv.reader(csvfile, delimiter=',')
    train_data = list(csv.reader(csvfile))
train_data = train_data[1:len(train_data )]    
train_data = np.asarray(train_data)

In [ ]:
SOPs_train_index= np.argsort(SOPs_train)
SOPs_train= SOPs_train[SOPs_train_index]
image_paths = image_paths[SOPs_train_index]


SOPs_in_traincsv_index =  np.argsort(train_data[:,2])
SOPs_in_traincsv = train_data[:,2][SOPs_in_traincsv_index]
y_train_images = train_data[:,3][SOPs_in_traincsv_index]

print(len(SOPs_train))
print(len(train_data))
print(len(y_train_images))

print(SOPs_in_traincsv[0])
print(image_paths[0])
print(SOPs_train[0])
print(SOPs_in_traincsv[len(SOPs_in_traincsv)-1])
print(SOPs_train[len(SOPs_train)-1])

# PREPROCESS IMAGES

In [ ]:

def preprocess_image(path):
    dataset = dcmread(path)
    image_dene = np.asarray(dataset.pixel_array)
    preprocessed_image= cv2.resize(image_dene , (150, 150),  interpolation = cv2.INTER_AREA)
    preprocessed_image = cv2.merge((preprocessed_image,preprocessed_image,preprocessed_image))
    preprocessed_image = tf.keras.applications.xception.preprocess_input(preprocessed_image) 

    return preprocessed_image
    
    
      
    



In [ ]:
y_train_images = y_train_images.astype(np.float)

# CREATE 2D MODEL FOR IMAGES

In [ ]:

METRICS = [
      #keras.metrics.TruePositives(name='tp'),
      #keras.metrics.FalsePositives(name='fp'),
      #keras.metrics.TrueNegatives(name='tn'),
      #keras.metrics.FalseNegatives(name='fn'), 
      #keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      #keras.metrics.Recall(name='recall'),
      #keras.metrics.AUC(name='auc'),
]
base_model = tf.keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False) 
base_model.trainable = False
inputs = keras.Input(shape=(150, 150, 3))

x = base_model(inputs, training=False)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
print(x.shape)
#x = tf.keras.layers.Dense(64, activation='relu')(x)
#x = tf.keras.layers.Dropout(0.5)(x)
print(x.shape)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model_for_pe_on_image = tf.keras.Model(inputs, outputs)
model_for_pe_on_image.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

#model_for_pe_on_image = keras.models.load_model('../input/model2d500/model_2d.h5')

# CREATE BALANCED DATASET

In [ ]:
for times in range(4):
    print("times :",times)
    for count in range(0,30):
        no_of_data_to_be_trained = 50000
        print("No of data that is trained",count*50000)
        start = count* no_of_data_to_be_trained 
        end = start + no_of_data_to_be_trained 
        train_labels_to_be_used = y_train_images[start:end]
        image_paths_to_be_used = image_paths[start:end]
        indexes_0 = np.where(train_labels_to_be_used== 0.0)[0]
        indexes_1 = np.where(train_labels_to_be_used == 1.0)[0]

        indexes_0_shuffled = np.random.shuffle(indexes_0)
        indexes_1_shuffled = np.random.shuffle(indexes_1)
        indexes_0_shuffled = indexes_0[0:2*int(len(indexes_1))]
        concat_indexes_shuffled = np.concatenate((indexes_0_shuffled,indexes_1),axis=None)
        np.random.shuffle(concat_indexes_shuffled)
        print(indexes_0.shape)
        print(indexes_1.shape)
        print(concat_indexes_shuffled.shape)
        shuffled_image_paths = image_paths_to_be_used[concat_indexes_shuffled]
        shuffled_y_train_images = train_labels_to_be_used[concat_indexes_shuffled]

        x_train_images = np.ones((len(shuffled_image_paths), 150, 150, 3),  dtype=np.int16)
        for i in range(0,len(shuffled_image_paths)):
            if i % 1000 == 0 :
                print(i) 
            x_train_images[i]= preprocess_image(shuffled_image_paths[i])

        model_for_pe_on_image.fit(x=x_train_images, y=shuffled_y_train_images, batch_size=64, epochs=1)
        model_for_pe_on_image.save('model_last.h5')  # creates a HDF5 file 'my_model.h5'



In [ ]:
"""
for count in range(4,8):
    print("No of data that is trained",count*100000)
    no_of_data_to_be_trained = 50000
    start = count*50000
    end = start + no_of_data_to_be_trained 
    train_labels_to_be_used = y_train_images[start:end]
    image_paths_to_be_used = image_paths[start:end]
    indexes_0 = np.where(train_labels_to_be_used== 0.0)[0]
    indexes_1 = np.where(train_labels_to_be_used == 1.0)[0]

    indexes_0_shuffled = np.random.shuffle(indexes_0)
    indexes_1_shuffled = np.random.shuffle(indexes_1)
    indexes_0_shuffled = indexes_0[0:int(3*len(indexes_1))]
    concat_indexes_shuffled = np.concatenate((indexes_0_shuffled,indexes_1),axis=None)
    np.random.shuffle(concat_indexes_shuffled)
    print(indexes_0.shape)
    print(indexes_1.shape)
    print(concat_indexes_shuffled.shape)
    shuffled_image_paths = image_paths_to_be_used[concat_indexes_shuffled]
    shuffled_y_train_images = train_labels_to_be_used[concat_indexes_shuffled]

    x_train_images = np.ones((len(shuffled_image_paths), 150, 150, 3),  dtype=np.int16)
    for i in range(0,len(shuffled_image_paths)):
        if i % 1000 == 0 :
            print(i) 
        x_train_images[i]= preprocess_image(shuffled_image_paths[i])

    model_for_pe_on_image.fit(x=x_train_images, y=shuffled_y_train_images, batch_size=64, epochs=5, validation_split=0.2)
model_for_pe_on_image.save('model_second.h5')  # creates a HDF5 file 'my_model.h5'
"""

In [ ]:
x_train_images = []

In [ ]:
#from keras.models import load_model
#model_for_pe_on_image.save('model_first.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
"""
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model_2d = Sequential()
model_2d.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3)))
model_2d.add(Activation('relu'))
model_2d.add(MaxPooling2D(pool_size=(2, 2)))

model_2d.add(Conv2D(32, (3, 3)))
model_2d.add(Activation('relu'))
model_2d.add(MaxPooling2D(pool_size=(2, 2)))

model_2d.add(Conv2D(64, (3, 3)))
model_2d.add(Activation('relu'))
model_2d.add(MaxPooling2D(pool_size=(2, 2)))


model_2d.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model_2d.add(Dense(64))
model_2d.add(Activation('relu'))
model_2d.add(Dropout(0.5))
model_2d.add(Dense(1))
model_2d.add(Activation('sigmoid'))

model_2d.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


model_2d.fit(x=x_train_images, y=y_train_images, batch_size=64, epochs=50, validation_split=0.2)
"""

# SIMPLY TRY DATA

In [ ]:
no_of_train_test = 2000
x_train_test = np.ones((no_of_train_test , 150, 150, 3),  dtype=np.int16)
for i in range(0,no_of_train_test ):
    if i % 1000 == 0 :
        print(i) 
    x_train_test[i]= preprocess_image(image_paths[i])

In [ ]:
pred = model_for_pe_on_image.predict(x_train_test)
results = model_for_pe_on_image.evaluate(x_train_test, y_train_images[0:no_of_train_test].astype(np.float), batch_size=64)
pred_binary = np.ones(no_of_train_test)
print(len(np.where(pred>0.5)[0]))
print(len(np.where(pred<=0.5)[0]))

pred_binary[np.where(pred>0.6)[0]] = 1
pred_binary[np.where(pred<=0.6)[0]] = 0
unique, counts = np.unique(pred_binary, return_counts=True)
print(unique, counts)

import sklearn 
print(sklearn.metrics.confusion_matrix(y_train_images[0:no_of_train_test],pred_binary,labels =[0,1]))
print(pred_binary)
print(y_train_images[0:no_of_train_test])

In [ ]:
pred_binary[np.where(pred>0.5)[0]] = 1
pred_binary[np.where(pred<=0.5)[0]] = 0
unique, counts = np.unique(pred_binary, return_counts=True)
print(unique, counts)

import sklearn 
print(sklearn.metrics.confusion_matrix(y_train_images[0:no_of_train_test],pred_binary,labels =[0,1]))
print(pred_binary)
print(y_train_images[0:no_of_train_test])

In [ ]:
label = y_train_images[0:no_of_train_test]
for i in range(0,2000):

    print("i :",i," pred: ",pred[i],"true label :",label[i])
#print(y_train_images[0:no_of_train_test])

In [ ]:
#unique, indexes, counts = np.unique(y_train_images[0:1000],return_index=True, return_counts=True,)
#print(unique, counts)
#print(indexes)
#indexes_0 = np.where(y_train_images[0:1000] == 0.0)[0]
#indexes_1 = np.where(y_train_images[0:1000] == 1.0)[0]

# SAVE MODEL

In [ ]:
#from keras.models import load_model
#model_for_pe_on_image.save('model_2d.h5')  # creates a HDF5 file 'my_model.h5'